In [ ]:
###################################
# ASV picking using DADA2 pipeline
###################################


In [11]:
# Prepare folder for dada2 output: 
dir.create("output/2_dada2")

dir.create("output/2_dada2/diagnostic_plots")

dir.create("output/2_dada2/objects")



In [12]:
# Load libraries 
source("scr/functions/general/load_abs_install_pkg.R")

load_abs_install_pkg(c("dada2", "phyloseq", "tidyverse"))



[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

In [13]:
# Make a list of fastq files in the directory 
path = "data/Dada2_input/"

files <- list.files(path)

files <- files[grep(".fastq", files)]



In [14]:
# Separate forward and revers reads into different variables 
fnFs <- sort(list.files(path, pattern="R1.fastq", full.names = TRUE))

fnRs <- sort(list.files(path, pattern="R2.fastq", full.names = TRUE))


In [15]:
# Make a variable containing only files names 
sample.names <- unique(gsub("_R1.fastq|_R2.fastq", "", files))


In [16]:
# Check reads quality by graphing 
#   Plots are saved in "output/dada2/diagnostic_plots"
forwReads <- plotQualityProfile(fnFs[1:4])

revReads <-plotQualityProfile(fnRs[1:4])

ggsave("output/2_dada2/diagnostic_plots/forward_reads_qualty.pdf", plot = forwReads )

ggsave("output/2_dada2/diagnostic_plots/revers_reads_qualty.pdf", plot = revReads )

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.
Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.
Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image


In [17]:
# Prepare directory and names for filtered files 
filtFs <- file.path(path, "filtered", paste0(sample.names, "_F_filt.fastq.gz"))

filtRs <- file.path(path, "filtered", paste0(sample.names, "_R_filt.fastq.gz"))

names(filtFs) <- sample.names

names(filtRs) <- sample.names

In [18]:
out <- filterAndTrim(fnFs, filtFs, fnRs, filtRs, 
                     truncLen=c(280, 240), # Truncet bad reads at the end of the reads 
                     # however keep in mind that reads should overlap at least 20bp
                     # in our case amplicon is 444 nucleotide long, so total length should be 464 at least. 
                     trimLeft = c(17, 21), # Trim primers from the begining of sequence                       
                     maxN=0, 
                     maxEE=c(1,1), 
                     truncQ=2, 
                     rm.phix=TRUE,
                     compress=TRUE, 
                     multithread=TRUE) #only for Linux  



Creating output directory: data/Dada2_input//filtered


In [20]:
# Check reads number before and after filtering by plotting (personal addition, no needed for DADA2)
#  Prepare data for the plot 
out.gg <- as.data.frame(out)

out.gg$SampleID <- rownames(out.gg)

out.gg.m <- reshape2::melt(out.gg)


# Plot using ggplot package 
filtReads <- ggplot(out.gg.m, aes(x=SampleID, y=value, color=variable, group=SampleID)) + 
             geom_point(size = 3.5) + 
             theme_bw() + 
             geom_line(size = 1) + 
             theme(axis.text.x = element_text(angle = 90, hjust = 1)) + 
             ylab("Reads number")

# Save the plot in output/dada2/diagnostic_plots/ folder 
ggsave("output/2_dada2/diagnostic_plots/n_reads_filter1.pdf", plot = filtReads, width = 12 )



Using SampleID as id variables
Saving 12 x 6.67 in image


In [21]:
# Learn errors in sequences for forward and reverse reads 
errF <- learnErrors(filtFs, multithread=TRUE, nbases = 2e8)

errR <- learnErrors(filtRs, multithread=TRUE, nbases = 2e8)

save(list = c("errF", "errR"), file="output/2_dada2/objects/error_rate_FR.RData")


204404389 total bases in 777203 reads from 26 samples will be used for learning the error rates.
208382223 total bases in 951517 reads from 31 samples will be used for learning the error rates.


In [22]:
errFplot <- plotErrors(errF, nominalQ=TRUE)

errRplot <- plotErrors(errR, nominalQ=TRUE)


# Save plots in output/dada2/diagnostic_plots/
ggsave("output/2_dada2/diagnostic_plots/error_rate_F.pdf", plot = errFplot)

ggsave("output/2_dada2/diagnostic_plots/error_rate_R.pdf", plot = errRplot)

Saving 6.67 x 6.67 in image
Warning message:
“Transformation introduced infinite values in continuous y-axis”Saving 6.67 x 6.67 in image
Warning message:
“Transformation introduced infinite values in continuous y-axis”

In [23]:
# Dereplication - combine identical sequences into unique with corresponding number 
#                    of reads 
derepFs <- derepFastq(filtFs, verbose=FALSE)

derepRs <- derepFastq(filtRs, verbose=FALSE)

# Name the derep-class objects by the sample names
names(derepFs) <- sample.names

names(derepRs) <- sample.names

# Save dereplicated objects 
save(derepFs, file="output/2_dada2/objects/derepFs.RData")

save(derepRs, file="output/2_dada2/objects/derepRs.RData")



In [24]:
# Samples inference 
#      *addiotion of (..., pool=TRUE) argument could increase sensetivity for rear variates 
dadaFs <- dada(derepFs, err=errF, multithread=TRUE)

dadaRs <- dada(derepRs, err=errR, multithread=TRUE)

# Back up generated objects 
save(dadaFs, file="output/2_dada2/objects/dadaFs.RData")

save(dadaRs, file="output/2_dada2/objects/dadaRs.RData")


Sample 1 - 25711 reads in 8609 unique sequences.
Sample 2 - 20402 reads in 10848 unique sequences.
Sample 3 - 21412 reads in 9209 unique sequences.
Sample 4 - 18693 reads in 8771 unique sequences.
Sample 5 - 42664 reads in 20084 unique sequences.
Sample 6 - 41697 reads in 19934 unique sequences.
Sample 7 - 18062 reads in 10909 unique sequences.
Sample 8 - 38713 reads in 19433 unique sequences.
Sample 9 - 39967 reads in 16433 unique sequences.
Sample 10 - 47249 reads in 23758 unique sequences.
Sample 11 - 42551 reads in 20342 unique sequences.
Sample 12 - 24232 reads in 11046 unique sequences.
Sample 13 - 22848 reads in 10319 unique sequences.
Sample 14 - 22982 reads in 10265 unique sequences.
Sample 15 - 51530 reads in 24836 unique sequences.
Sample 16 - 16 reads in 13 unique sequences.
Sample 17 - 40261 reads in 18651 unique sequences.
Sample 18 - 19352 reads in 10054 unique sequences.
Sample 19 - 45853 reads in 22904 unique sequences.
Sample 20 - 40695 reads in 19659 unique sequences

In [25]:
# Merge paired reads 
mergers <- mergePairs(dadaFs, derepFs, dadaRs, derepRs, verbose=TRUE)

# Back up generated object 
save(mergers, file="output/2_dada2/objects/mergers.RData")

20673 paired-reads (in 776 unique pairings) successfully merged out of 23905 (in 2925 pairings) input.
12067 paired-reads (in 577 unique pairings) successfully merged out of 17082 (in 3906 pairings) input.
14644 paired-reads (in 693 unique pairings) successfully merged out of 18786 (in 3442 pairings) input.
12356 paired-reads (in 549 unique pairings) successfully merged out of 16086 (in 3054 pairings) input.
27032 paired-reads (in 1129 unique pairings) successfully merged out of 37110 (in 7969 pairings) input.
27274 paired-reads (in 1213 unique pairings) successfully merged out of 36450 (in 7165 pairings) input.
9906 paired-reads (in 513 unique pairings) successfully merged out of 14484 (in 3661 pairings) input.
23642 paired-reads (in 909 unique pairings) successfully merged out of 32945 (in 6921 pairings) input.
28311 paired-reads (in 1368 unique pairings) successfully merged out of 35568 (in 5935 pairings) input.
28507 paired-reads (in 1263 unique pairings) successfully merged out of

In [26]:
# Construct sequence table
seqtab <- makeSequenceTable(mergers)

# Check visually reads distribution by their merged length
reads.per.seqlen <- tapply(colSums(seqtab), factor(nchar(getSequences(seqtab))), sum)

df <- data.frame(length=as.numeric(names(reads.per.seqlen)), count=reads.per.seqlen)

readsDisr <- ggplot(data=df, aes(x=length, y=count)) + geom_col() + theme_bw()

# Save the plot 
ggsave("output/2_dada2/diagnostic_plots/leng_reads_distr.pdf", plot = readsDisr)

# Trim reads outside of reads distribution borders 
seqtab2 <- seqtab[,nchar(colnames(seqtab)) %in% seq(400, 430)]


Saving 6.67 x 6.67 in image


In [27]:
# Remove chimeras (will be majority of variants)
seqtab.nochim <- removeBimeraDenovo(seqtab2, method="consensus", multithread=TRUE, verbose=TRUE)

# Save ASV as R object and csv table 
save(seqtab.nochim, file = "output/2_dada2/objects/seqtab_nochim.RData")

write.csv(seqtab.nochim, file = "output/2_dada2/seqtab_nochim.csv")

Identified 33182 bimeras out of 48198 input sequences.


In [28]:
# Check how many sequenses get through 
getN <- function(x) sum(getUniques(x))

track <- cbind(out, sapply(dadaFs, getN), sapply(dadaRs, getN), 
               sapply(mergers, getN), rowSums(seqtab.nochim))
# If processing a single sample, remove the sapply calls: e.g. 
# replace sapply(dadaFs, getN) with getN(dadaFs)

colnames(track) <- c("input", "filtered", "denoisedF", "denoisedR", "merged", "nonchim")

rownames(track) <- sample.names

write.csv(track, file = "output/2_dada2/diagnostic_plots/track.csv")

In [29]:
# Taxonomy assignment  
#  Assign taxonomy using classical method 
taxa <- assignTaxonomy(seqtab.nochim, "resourses/silva_nr_v138_train_set.fa.gz",
                       multithread=TRUE)

# Save taxonomy table as csv and R object
write.csv(x = taxa, "output/2_dada2/taxa.csv")

save(taxa, file ="output/2_dada2/objects/taxa.RData")

# Assign taxonomy to species level using exact match 
# taxa.species <- addSpecies(taxa, "resourses/silva_species_assignment_v138.fa.gz")

In [31]:
# Read in metadata for samples 
metadata_final <- read.csv("output/1_shedding_analysis/metadata_f.csv")

rownames(metadata_final) <- metadata_final$NewId

In [32]:
rownames(seqtab.nochim) <- gsub("\\.gz", "", rownames(seqtab.nochim))


In [33]:
# Combine taxonomic table, ASV count table and metadata into a pholoseq object. 
ps <- phyloseq(otu_table(seqtab.nochim, taxa_are_rows=FALSE), 
               sample_data(metadata_final), 
               tax_table(taxa))

# Save phyloseq object 
save(ps, file = "output/2_dada2/phyloseq0.RData")
saveRDS(ps, "output/2_dada2/phyloseq0.rds")